In [18]:
import pandas as pd
import os
pd.options.mode.chained_assignment = None

In [98]:
# function to clean data of data frame
def clean_df(df):
    try:
        df.columns = ['Party', 'Member', 'Accomodation - Wellington',
        'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
        'VIP Transport', 'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending']
    except:
        df.columns = ['Party', 'Member', 'Accomodation - Wellington',
        'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
        'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending']
        df.insert(6, "VIP Transport", "")


    df['Party'] = df['Party'].fillna(method='ffill')
    df = df[df["Party"].str.contains("Total|[0-9]") == False]
    df = df[df['Member'].notna()]


    df['Member'] = df['Member'].str.replace(' \([0-9]\)', '', regex=True)
    return df

In [20]:
# function to add column with the quarter
def append_quarter(filename, df):
    if filename.split()[-1][0].isnumeric():
        file_quarter = str(filename.split()[-2]+" "+filename.split()[-1]).replace('.xlsx', '')
    else:
        file_quarter = str(filename.split()[-1]+" "+filename.split()[0]).replace('.xlsx', '')
    df['Quarter Ending'] = file_quarter
    return df

In [99]:
# Loop through files in Downloads folder, import clean and append to one file

final_df = pd.DataFrame([], columns=['Party', 'Member', 'Accomodation - Wellington',
    'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
    'VIP Transport', 'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending'])

for filename2 in os.listdir("Downloads/"):
    if filename2.endswith(".xlsx") and filename2.startswith("2"): 
         df = pd.read_excel ('Downloads/'+filename2, skiprows=2, header=[0,1])
         df = append_quarter(filename2, df)
         df = clean_df(df)
         final_df = pd.concat([df, final_df])
    else:
        pass



In [100]:
def insert_comma(string):
    string = str(string)
    return string[:string.rfind(" ")] + "," + string[string.rfind(" "):]


In [101]:
final_df['Member'] = final_df['Member'].str.replace('*', '', regex=True)
final_df['Member'] = final_df['Member'].str.replace('\\n', '', regex=True)
final_df['Member'] = final_df['Member'].str.strip()
final_df['Member'] = final_df['Member'].str.replace('Te Ururoa', 'Te_Ururoa', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Kanwaljit ,Singh', 'Kanwaljit Singh', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Kanwaljit Singh', 'Kanwaljit_Singh', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Julie Anne', 'Julie-Anne', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Gharahman', 'Ghahraman', regex=True)
final_df['Member'] = final_df['Member'].str.replace(' ,', ',', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Anae Neru', 'Anae_Neru', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Aupito William', 'Aupito_William', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Su\'a, William', 'Aupito_William', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Turei', 'Tūrei', regex=True)
final_df['Member'] = final_df['Member'].str.replace('van, de Molen Tim ', 'van de Molen, Tim', regex=True)
final_df['Member'] = final_df['Member'].str.replace(',', ', ', regex=True)
final_df['Member'] = final_df['Member'].str.replace('  ', ' ', regex=True)
final_df['Member'] = final_df['Member'].apply(insert_comma)
final_df['Member'] = final_df['Member'].str.replace(',,', ',', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Warren-Clarke, Angie', 'Warren-Clark, Angie', regex=True)



In [109]:
final_df.loc[:, 'Accomodation - Wellington':'Inter-Parliamentary Office of the Clerk']

,Accomodation - Wellington,Accomodation - Non-Wellington,Air Travel,Surface Travel,VIP Transport,Grand Total,Inter-Parliamentary Office of the Clerk
1,6790.0,490.25,7800.76,5559.37,,20640.38,-
4,8323.28,1379.57,12049.96,5766.34,,27519.15,-
5,0.0,0.0,0.0,0.0,,0.0,-
6,9402.16,391.13,8678.12,6603.87,,25075.28,-
7,0.0,1887.16,5842.08,3709.92,,11439.16,4979
...,...,...,...,...,...,...,...
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,5826.0,392.0,1821.0,3577.0,0.0,11616.0,0.0
127,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128,8323.0,705.0,2895.0,993.0,0.0,12917.0,0.0


In [102]:
final_df.to_excel('nz_mp_expenses.xlsx')